In [118]:
import numpy as np
import pandas as pd
import re

import ast

from collections import Counter
import string

from konlpy.tag import Kkma
from konlpy.tag import Mecab
from konlpy.utils import pprint
from konlpy.tag import Twitter,Okt

import kss

import os

from tqdm import tqdm

from gensim.models import Word2Vec
import random

import time

import threading

from dateutil.parser import parse
from datetime import datetime

import plotly.express as px
import plotly.graph_objects as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

In [2]:
okt = Okt()

# data

In [3]:
def data_list(path_):
    #path_ = r"C:\Users\Samsung\jupyter\Project_Deep_learning_2020\data\ear_phone"
    file_list = os.listdir(path_)
    file_list_xlsx_all = [path_+"\\"+f for f in file_list if f.endswith(".xlsx")]
    file_list_xlsx = [f for f in file_list if f.endswith(".xlsx")]
    return file_list_xlsx,file_list_xlsx_all

In [4]:
def load_data(path_):
    data_ = pd.read_excel(path_).drop(columns="Unnamed: 0")
    return data_

In [5]:
def load_all(ta,ra):
    da_1 = pd.DataFrame()
    for i in ta[1][ra[0]:ra[1]]:
        da_ = load_data(i)
        da_1 = pd.concat([da_1,da_]).reset_index(drop=True)
    return da_1

In [6]:
def parsing_date(df_):
    #df_ = coo_test
    parsing = [parse(d) for d in df_.loc[:,"date"]]
    df_["parse_date"] = parsing
    return df_

In [7]:
def prepro_date(df_):
    df_["year"] = [d.strftime("%Y") for d in df_["parse_date"]]
    df_["month"] = [d.strftime("%Y-%m") for d in df_["parse_date"]]
    df_["day"] = [d.strftime("%Y-%m-%d") for d in df_["parse_date"]]
    return df_

In [8]:
def remove_na(df_,target_):
    df_2 = df_[df_[target_] != "None"]
    return df_2.reset_index(drop=True)

In [9]:
def sen_split(df):
    sent_li = []
    for i in tqdm(range(len(df))):
        try:
            sen_ = kss.split_sentences(df["contents"][i])
            sent_li.append(sen_)
        except:
            sent_li.append([])
            print(df["contents"][i])
    df.loc[:,"sentences"] = sent_li
    return df

In [10]:
def pos_split(df):
    tag_li = []
    wo_li = []
    for i in tqdm(range(len(df))):
        sen = df["sentences"][i]
        tags_ = []
        wos_ = []
        for s in sen:
            tag_ = okt.pos(s,stem=True)
            word_ = [w[0] for w in tag_]
            tags_.append(tag_)
            wos_.append(word_)
        tag_li.append(tags_)
        wo_li.append(wos_)
    df.loc[:,"sen_tag"] = tag_li
    df.loc[:,"sen_word"] = wo_li
    return df

In [11]:
def all_pro(df):
    df2 = prepro_date(parsing_date(df))
    df3 = remove_na(df2,"contents")
    print("sentence_split")
    df4 = sen_split(df3)
    print("pos_tagging")
    df5 = pos_split(df4)
    return df5

# dic

In [549]:
tag_dic = {
    "Noun":["NNG","NNP","NNM","Noun"],
    "Verb":['VV', 'VA', 'VXV', 'VCN',"Verb"],
    "Adverb":["MAG","MAC","Adverb"],
    "Adjective":["XPV","XSM","XSO","XR","Adjective"],
    "Unknown":['UN', 'UV', 'UE',"Unknown"],
    "ETC":["EMO","OL","ETC"]
}

In [550]:
target_  = ["Noun","Adjective","Verb","Adverb"]

In [551]:
tag_dic_2 = {}
for key,value in tag_dic.items():
    for va in value:
        tag_dic_2[va] = key
tag_dic_2

{'NNG': 'Noun',
 'NNP': 'Noun',
 'NNM': 'Noun',
 'VV': 'Verb',
 'VA': 'Verb',
 'VXV': 'Verb',
 'VCN': 'Verb',
 'MAG': 'Adverb',
 'MAC': 'Adverb',
 'XPV': 'Adjective',
 'XSM': 'Adjective',
 'XSO': 'Adjective',
 'XR': 'Adjective',
 'UN': 'Unknown',
 'UV': 'Unknown',
 'UE': 'Unknown',
 'EMO': 'ETC',
 'OL': 'ETC'}

In [576]:
def extract_pos(li_,version,target_,tag_dic_2):
    if version == "okt":
        return [f[0] for f in li_ if f[1] in target_]
    elif version == "kkma":
        at = []
        for f in li_:
            try:
                if tag_dic_2[f[1]] in target_:
                    at.append(f[0])
            except:
                pass
        return at

In [583]:
def Pos_tag(pos_,version,target_,tag_dic_2):
    def extract_pos(li_,version,target_,tag_dic_2):
        if version == "okt":
            return [f[0] for f in li_ if f[1] in target_]
        elif version == "kkma":
            at = []
            for f in li_:
                try:
                    if tag_dic_2[f[1]] in target_:
                        at.append(f[0])
                except:
                    pass
            return at
    if version == "okt":
        pos_tag = [ okt.pos(f,stem=True) for f in pos_]
        pos_ex = [extract_pos(f,version,target_,tag_dic_2) for f in pos_tag]
    elif version == "kkma":
        pos_tag = [kkma.pos(f) for f in pos_]
        pos_ex = [extract_pos(f,version,target_,tag_dic_2) for f in pos_tag]
    return pos_ex

In [584]:
pos_ = dic_["긍정"][1].split(",")
pos_

['바람세기 조절도 가능하고요',
 ' 강약 조절도 잘 되는거 같습니다',
 ' 3단계 버튼 조절이 가능하고',
 ' 바람세기도 조절이 가능하니 좋고',
 ' 풍량조절기도 잘되고',
 ' 바람세기 조절도 여러가지라서 사용하기 정말 편합니다',
 ' 바람 세기가 세분화되어 있어서 매우 편리하고 좋습니다',
 ' 강도조절이 가능하네요',
 ' 풍량 조절과 좀 더 드라이의 단계가 디테일해서 머리드라이할때 좀더 좋은것 같아요',
 ' 단계별로 바람세기를 조절해서 너무 좋다',
 ' 세기까지 다채로운 바람조절 가능',
 ' 바람세기를 삼단으로 섬세하게 조절 가능',
 ' 바람세기 조절이 간단하네요',
 '  바람세기가 완전 맘에 들어요']

In [585]:
Pos_tag(pos_,"kkma",["Noun","Adjective","Verb","Adverb"],tag_dic_2)

[['바람세기', '조절', '가능'],
 ['강약', '조절', '잘', '되', '같'],
 ['단계', '버튼', '조절', '가능'],
 ['바람세기', '조절', '가능', '좋'],
 ['풍', '량', '조절기', '잘되'],
 ['바람세기', '조절', '사용', '정말', '편하'],
 ['바람', '세기', '세분화', '있', '매우', '편리', '좋'],
 ['강도', '조절', '가능'],
 ['풍',
  '량',
  '조절',
  '좀',
  '더',
  '드라이',
  '단계',
  '디테일',
  '머리',
  '드라이',
  '때',
  '좀더',
  '좋',
  '같'],
 ['단계별', '바람세기', '조절', '너무', '좋'],
 ['세기', '다채', '바람', '조절', '가능'],
 ['바람세', '기르', '삼단', '섬세', '조절', '가능'],
 ['바람세기', '조절', '간단'],
 ['바람세기', '완전', '맘', '듣']]

In [561]:
def dic_tag2(dic_,version,target_,tag_dic_2=None):
    all_dic = {}
    for i in tqdm(range(len(dic_))):
        pos_all = []
        pos_noli = []
        neg_all = []
        neg_noli = []
        try:
            pos_ = dic_["긍정"][i].split(",")
            pos_ex = Pos_tag(pos_,version,target_,tag_dic_2)
            pos_no = Pos_tag(pos_,version,"Noun",tag_dic_2)
            
            [pos_noli.extend(w) for w in pos_no]
            [pos_all.extend(w) for w in pos_ex]
        except:
            pos_ex = []
            [pos_all.extend(w) for w in pos_ex]
        try:
            neg_ = dic_["부정"][i].split(",")
            neg_ex = Pos_tag(neg_,version,target_,tag_dic_2)
            neg_no = Pos_tag(neg_,version,"Noun",tag_dic_2)
            
            [neg_noli.extend(w) for w in neg_no]
            [neg_all.extend(w) for w in neg_ex]
        except:
            neg_ex = []
            [neg_all.extend(w) for w in neg_ex]
        
        
        try:
            fun = list(map(lambda x :x.strip(),dic_["attr_sim"][i].split("#")[1:]))
            fun.append(dic_["attr"][i].strip())

            fun = list(set(fun))
            funda = [f.split(" ") for f in fun]
            
            combi_ = [okt.pos(w,stem=True) for w in ast.literal_eval(dic_["attr_combi"][i])]
            combi_2 = [list(map(lambda x :x[0],w)) for w in combi_]
            funda.extend(combi_2)
            funda = delete_duple(funda)
            #funda = list(set(funda))
        except:
            funda = []
        kee = [[ke[0] for ke in okt.pos(dic_["key"][i],stem=True)]]
        noun_inter = [[ni] for ni in list(set(pos_noli).intersection(set(neg_noli)))]
        noun_union = [[ni] for ni in list(set(pos_noli).union(set(neg_noli)))]
        noun_union = delete_duple(noun_union)
        all_ = {"pos":pos_ex,"neg":neg_ex,"noun_inter":noun_inter,"noun_union":noun_union,"funda":funda,"key":kee}
        all_dic[dic_["attr"][i]] = all_

    return all_dic

In [600]:
def removel_list(Tester_,removal):
    for re in removal:
        try:
            Tester_.remove(re)
        except:
            pass
    return Tester_

In [601]:
def extract_pos(li_,target_):
    return [f[0] for f in li_ if f[1] in target_]


In [643]:
def dic_tag(dic_,target_):
    all_dic = {}
    all_dic_2 = {}
    for i in tqdm(range(len(dic_))):
        pos_all = []
        pos_noli = []
        neg_all = []
        neg_noli = []
        # pos 는 target으로 나눈것
        # pos 
        try:
            pos_ = dic_["긍정"][i].split(",")
            pos_tag = [ okt.pos(f,stem=True) for f in pos_]
            pos_ex = [extract_pos(f,target_) for f in pos_tag]

            pos_no = [extract_pos(f,"Noun") for f in pos_tag]
            [pos_noli.extend(w) for w in pos_no]

            [pos_all.extend(w) for w in pos_ex]

        except:
            pos_ex = []
            [pos_all.extend(w) for w in pos_ex]
        try:
            neg_ = dic_["부정"][i].split(",")
            neg_tag = [ okt.pos(f,stem=True) for f in neg_]

            neg_no = [extract_pos(f,"Noun") for f in neg_tag]
            [neg_noli.extend(w) for w in neg_no]

            neg_ex = [extract_pos(f,target_) for f in neg_tag]
            [neg_all.extend(w) for w in neg_ex]
        except:
            neg_ex = []
            [neg_all.extend(w) for w in neg_ex]
        
        
        try:
            fun = list(map(lambda x :x.strip(),dic_["attr_sim"][i].split("#")[1:]))
            fun.append(dic_["attr"][i].strip())

            fun = list(set(fun))
            funda = [f.split(" ") for f in fun]
            
            combi_ = [okt.pos(w,stem=True) for w in ast.literal_eval(dic_["attr_combi"][i])]
            combi_2 = [list(map(lambda x :x[0],w)) for w in combi_]
            funda.extend(combi_2)
            funda = delete_duple(funda)
            #funda = list(set(funda))
        except:
            funda = []
        kee = [[ke[0] for ke in okt.pos(dic_["key"][i],stem=True)]]
        kee2 = []
        for ke in kee:
            for ke2 in ke:
                kee2.append([ke2])
                
        removal2 = [['기능'],['감'], ['/'], ['및']]
        kee2 = removel_list(kee2,removal2)
        noun_inter = [[ni] for ni in list(set(pos_noli).intersection(set(neg_noli)))]
        noun_union = [[ni] for ni in list(set(pos_noli).union(set(neg_noli)))]
        noun_union = delete_duple(noun_union)
        
        removal = [['것'],['이'], ['왜'],['더']]
        
        pos_ex = removel_list(pos_ex,removal)
        neg_ex = removel_list(neg_ex,removal)
        noun_inter = removel_list(noun_inter,removal)
        noun_union = removel_list(noun_union,removal)
        funda = removel_list(funda,removal)
        kee = removel_list(kee,removal)
        
        
        all_ = {"pos":pos_ex,"neg":neg_ex,"noun_inter":noun_inter,"noun_union":noun_union,"funda":funda,"key":kee,"key2":kee2}
        all_2 = {"pos":list(set(pos_all)),"neg":list(set(neg_all))}
        all_dic[dic_["attr"][i]] = all_
        all_dic_2[dic_["attr"][i]] = all_2
    return all_dic,all_dic_2

In [438]:
def match_f(x,se_t):
    if all(elem in se_t for elem in x):    
        return 1,x
    else:
        return 0,"None"

In [612]:
def match_result(dic_list,check_list):
    
    def match_f(x,se_t):
        if all(elem in se_t for elem in x):    
            return 1,x
        else:
            return 0,[""]
    
    score = list(map(lambda x:match_f(x,check_list)[0] ,dic_list))
    matching = list(map(lambda x:match_f(x,check_list)[1] ,dic_list))
    match_li = []
    [match_li.extend(em) for em in matching]
    return sum(score)," ".join(match_li)

In [515]:
def delete_duple(list_):
    tass = []
    for i in list_:
        if i in tass:
            pass
        else:
            tass.append(i)
    return tass

In [537]:
okt = Okt()
kkma = Kkma()

In [516]:
set(["a","b"]) in set(["b","c","a",""])

False

In [495]:
["b"] in ["b","c"]

False

In [442]:
all(elem in ["b","c"] for elem in ["b","a"])

False

In [443]:
okt.pos("누구 없냐",stem=True)

[('누구', 'Noun'), ('없다', 'Adjective')]

# Dic

In [1]:
dic_ = pd.read_excel(r"C:\Users\Samsung\Desktop\Project\이희정 교수님\project\dic_data\드라이기_20201207.xlsx",sheet_name="data_")

NameError: name 'pd' is not defined

In [564]:
dic_ = dic_.rename(columns={"Unnamed: 0":"key","Product Attribute 측면 속성":"attr","유사 키워드(attr_sim)":"attr_sim","토큰화 방식으로 각 속성을 나타내는 키워드 조합":"attr_combi"})

In [565]:
dic_.head()

,key,attr,attr_sim,attr_combi,count,긍정,부정
0,바람,바람 세기,#바람 강도 #바람 파워 #드라이기 파워 #건조 속도,"['바람 세기', '바람 강하다', '바람 세다', '바람 쎄다', '바람 약하다'...",20916,"바람도 쎄고, 드라이기 바람세기 만족해요, 바람도 세서 좋아요, 바람세기도 좋아서...","바람이 약해요, 바람도 세지도 않아요, 생각만큼 바람이 쎄지는 않습니다, 바람이 흩..."
1,바람,풍량 조절,#풍속 조절 #바람강도 조절 #강도 조절 #바람 세기 조절,"['풍량 조절', '풍속 조절', '바람 강도 조절', '강도 조절', '바람 세기...",1002,"바람세기 조절도 가능하고요, 강약 조절도 잘 되는거 같습니다, 3단계 버튼 조절이 ...","단계조절이 안되는 것같아서, 풍량조절부분이 뻑뻑하네요, 바람 세기 조절이 강약 뿐인..."
2,바람,바람 회전,#토네이도 #회전 바람,"['바람 회전', '토네이도']",28,바람 나오는 부위도 360회전이 되니 사용할때 내가 편한대로 할수 잇어서 더 좋아여...,괴물 토네이도 이런 표현은좀 오바네요
3,온도,냉풍,#차가운 바람 #찬바람,"['냉풍', '차가운 바람', '찬바람', '바람 차갑다']",2476,"가장 큰 장점은 냉풍이에요, 냉풍이라 머릿결도 상하지 않을거 같고 너무 좋네요, ...","냉풍은 더운바람 나옵니다, 냉풍이라고하지만 약간 따뜻한 바람이나오네요, 냉풍이 별로..."
4,온도,온풍,#따뜻한 바람 #부드러운 바람,"['온풍', '따뜻하다 바람', '부드럽다 바람']",2529,"파워 온풍, 따뜻한 바람이 잘 나와서 좋네요, 따뜻한 바람 강력, 따뜻한 바람 적...","온풍으로는 두피나 헤어 손상이 걱정됩니다, 너무 뜨거운듯, 따뜻한 바람을 별로 좋아..."


In [447]:
i = 1

fun = list(map(lambda x :x.strip(),dic_["attr_sim"][i].split("#")[1:]))
fun.append(dic_["attr"][i].strip())
#fun.append([dic_["attr_combi"][i]])
fun.extend(ast.literal_eval(dic_["attr_combi"][i]))
fun = list(set(fun))
fun

['바람 강도 조절', '풍량 조절', '바람 세기 조절', '풍속 조절', '강도 조절', '바람강도 조절']

In [448]:
[list(map(lambda x : x[0],okt.pos(w,stem=True))) for w in ast.literal_eval(dic_["attr_combi"][i])]

[['풍량', '조절'],
 ['풍속', '조절'],
 ['바람', '강도', '조절'],
 ['강도', '조절'],
 ['바람', '세기', '조절']]

In [449]:
a_dic["바람 세기"]["funda"]

[['바람', '파워'],
 ['바람', '세기'],
 ['드라이기', '파워'],
 ['바람', '강도'],
 ['건조', '속도'],
 ['바람', '강하다'],
 ['바람', '세다'],
 ['바람', '쎄다'],
 ['바람', '약하다'],
 ['건조', '빠르다'],
 ['건조', '느리다'],
 ['건조', '빨르다']]

In [450]:
i = 0

combi_ = [okt.pos(w,stem=True) for w in ast.literal_eval(dic_["attr_combi"][i])]
combi_2 = [list(map(lambda x :x[0],w)) for w in combi_]
combi_2

[['바람', '세기'],
 ['바람', '강하다'],
 ['바람', '세다'],
 ['바람', '쎄다'],
 ['바람', '약하다'],
 ['바람', '파워'],
 ['건조', '속도'],
 ['건조', '빠르다'],
 ['건조', '느리다'],
 ['건조', '빨르다']]

In [451]:
combi_[0][0]

('바람', 'Noun')

In [644]:
#a_dic,a_dic_2,all_inter = dic_tag2(dic_,["Noun","Adjective","Verb","Adverb"])
a_dic,a_dic_2 = dic_tag(dic_,["Noun","Adjective","Verb","Adverb"])
a_dic

100%|█████████████████████████████████████████████████████████████████████████████████| 70/70 [00:00<00:00, 107.22it/s]


{'바람 세기': {'pos': [['바람', '쎄다'],
   ['드라이기', '바람', '세기', '만족하다'],
   ['바람', '세다', '좋다'],
   ['바람', '세기', '좋다'],
   ['바람', '세기', '맘에듭니'],
   ['바람', '세기', '강하다'],
   ['강하다', '바람', '젖다', '머리', '금방', '말', '좋다'],
   ['바람', '세다', '빨리', '말르다'],
   ['사용', '하다', '적당하다', '바라다', '이다'],
   ['바람', '세기', '충분하다'],
   ['바람', '세기', '단계', '별로', '있다'],
   ['바람', '쎄다', '좋다'],
   ['바람', '세기', '최고', '이다'],
   ['바람', '세다', '빨리', '건조', '되다', '만족하다'],
   ['바람', '쎄다', '물기', '금방', '마르다', '느낌'],
   ['바람', '세기', '흡족하다'],
   ['바람', '쎄다', '따라오다', '힘들껍니'],
   ['바람', '최상급'],
   ['강력하다', '바라다', '이다'],
   ['빠르다', '확실하다', '건조', '가능하다'],
   ['바람', '세기', '엄청나다'],
   ['빠르다', '건조', '만족하다'],
   ['출력', '그렇다', '바람', '완전', '쎄다'],
   ['파워', '풀', '바람', '좋다'],
   ['바람', '세기', '최고', '진짜', '속', '시원하다', '나오다'],
   ['바람', '세기', '장난', '아니다', '쩌렁'],
   ['바람', '세기', '더하다', '나위', '없다'],
   ['바람', '세기', '작살', '여'],
   ['역시', '가용', '바람', '세기', '다르다'],
   ['가정', '용', '확실하다', '쎄다', '미용실', '정도', '바람', '나오다'],
   ['바람', '세기', '남다름']],
  'neg': [

In [588]:
a_dic2 = dic_tag2(dic_,"kkma",["Noun","Adjective","Verb","Adverb"],tag_dic_2)
a_dic2

100%|██████████████████████████████████████████████████████████████████████████████████| 70/70 [00:12<00:00,  5.68it/s]


{'바람 세기': {'pos': [['바람', '쎄'],
   ['드라이기', '바람세기', '만족'],
   ['바람', '세서', '좋'],
   ['바람세기', '좋'],
   ['바람세기', '맘', '드'],
   ['바람세기', '강하'],
   ['강', '바람', '젖', '머리', '금방', '마르', '좋'],
   ['바람', '세서', '빨리', '마르'],
   ['사용', '적당', '바람세기'],
   ['바람', '세기', '충분'],
   ['바람세기', '단계별', '있'],
   ['라', '바람', '쎄', '좋'],
   ['바람세기', '최고'],
   ['바람', '세서', '빨리', '건조', '만족'],
   ['바람', '쎄', '물기', '금방', '마르', '느낌'],
   ['바람세기', '흡족'],
   ['바람', '쎄기', '따라오', '힘들'],
   ['바람', '최상급'],
   ['강력', '바람세기'],
   ['빠르', '확실', '건조', '가능'],
   ['바람세기', '엄청나'],
   ['빠르', '건조', '만족'],
   ['고출력', '그렇', '바람', '완전'],
   ['파워', '풀', '바람', '좋'],
   ['바람세기', '최고', '진짜', '속', '시원', '나오'],
   ['바람세기', '장난', '아', '쩌렁'],
   ['바람세기', '더하', '없'],
   ['바람세기', '작살'],
   ['역시', '전문가', '바람세기', '다르'],
   ['가정용', '확실히', '쎄', '미용실', '정도', '바람', '나오'],
   ['바람세기', '남다르']],
  'neg': [['바람', '약하'],
   ['바람', '세지', '않'],
   ['생각', '바람', '쎄', '않'],
   ['바람', '흩어지', '느낌'],
   ['바람', '쐬', '약하'],
   ['바람', '강도', '조금', '약'],
   ['바람세기', '나

In [ ]:
dic_,version,target_,tag_dic_2=None

## matching의 기준은 list 와 list 로 하자 

- ex) ["바람","세기"] 와 ["바람","세기","약하다"]

In [518]:
sens = data_2.loc[0,"sen_word"]
sens

[['가격', '저렴하다', '간단하다', '손쉽다', '사용', '하다', '아주', '좋다'],
 ['남자', '들', '사용', '하다', '아주', '굿', '이다']]

In [671]:
def test_maker(testsen):
    a2 = okt.pos(testsen,stem=True)
    return [a[0] for a in a2]

In [528]:
sens = test_maker("가격은 정말 저렴하지만, 무거운 것 같아요")
sens

[['가격', '은', '정말', '저렴하다', ',', '무겁다', '것', '같다']]

In [606]:
sens = test_maker("두피 마사지 기능 정말 마음에 드네요")
sens

[['두피', '마사지', '기능', '정말', '마음', '에', '드네', '요']]

In [525]:
sens = data_2.loc[3,"sen_word"]
sens

[['저', '는', '아직도', '잘쓰다', '있다'], ['산지', '좀', '되어다', 'ㅛ', '는데도', '좋다'], ['가볍다']]

In [526]:
sens = data_2.loc[2,"sen_word"]
sens

[['가격', '대비', '만족하다', '.']]

In [527]:
sens = data_2.loc[4,"sen_word"]
sens

[['약간', '타다', '조금', '나요']]

In [672]:
sens = test_maker("두피 마사지 기능 정말 마음에 들고 가격도 양호하고 무게도 가볍고 좋네요")

In [680]:
sens = test_maker("뭐 마음에 드네 기능 가격이 무겁")

In [683]:
dic_1 = a_dic
for sen in sens:
    tag_li = []
    for key,value in dic_1.items():
        
        sco_1 = match_result(value["funda"],sen)
        sco_2 = match_result(value["noun_inter"],sen)
        sco_3 = match_result(value["key"],sen)
        sco_3_2 = match_result(value["key2"],sen)
        sco_4 = match_result(value["pos"],sen)
        sco_5 = match_result(value["neg"],sen)
        
        score_ = [sco_1[0],sco_2[0],sco_3[0],sco_3_2[0],sco_4[0],sco_5[0]]
        string_ = [sco_1[1],sco_2[1],sco_3[1],sco_3_2[1],sco_4[1],sco_5[1]]
        if score_[0] != 0:
            print(key)
            print(score_)
            print(score_[0])
            print(string_[0])
            print()
        elif score_[1] != 0:
            
            print(key)
            print(sen)
            print(score_)
            print(score_[1])
            print(string_[1])
            print()
        elif score_[2] != 0:
            print(score_)
            print(key)
            print(sen)
            print(score_[2])
            print(string_[2])
            print()

가성비
가격
[0, 1, 0, 1, 1, 1]
1
가격 

무게
[1, 0, 0, 0, 1, 1]
1
  무겁다 



In [681]:
dic_1 = a_dic
sen = sens
tag_li = []
for key,value in dic_1.items():

    sco_1 = match_result(value["funda"],sen)
    sco_2 = match_result(value["noun_inter"],sen)
    sco_3 = match_result(value["key"],sen)
    sco_3_2 = match_result(value["key2"],sen)
    sco_4 = match_result(value["pos"],sen)
    sco_5 = match_result(value["neg"],sen)

    score_ = [sco_1[0],sco_2[0],sco_3[0],sco_3_2[0],sco_4[0],sco_5[0]]
    string_ = [sco_1[1],sco_2[1],sco_3[1],sco_3_2[1],sco_4[1],sco_5[1]]
    i = 0
    if score_[i] != 0:
        tag_li.append((key,i,score_))
    else:
        i += 1
        while score_[i] != 0:
            tag_li.append((key,i,score_))
            i += 1
tag_li

[('무게', 0, [1, 0, 0, 0, 1, 1]), ('가성비', 1, [0, 1, 0, 1, 1, 1])]

In [690]:
def model1(dic_1,sen):
    tag_li = []
    for key,value in dic_1.items():

        sco_1 = match_result(value["funda"],sen)
        sco_2 = match_result(value["noun_inter"],sen)
        sco_3 = match_result(value["key"],sen)
        sco_3_2 = match_result(value["key2"],sen)
        sco_4 = match_result(value["pos"],sen)
        sco_5 = match_result(value["neg"],sen)

        score_ = [sco_1[0],sco_2[0],sco_3[0],sco_3_2[0],sco_4[0],sco_5[0]]
        string_ = [sco_1[1],sco_2[1],sco_3[1],sco_3_2[1],sco_4[1],sco_5[1]]
        i = 0
        if score_[i] != 0:
            tag_li.append((key,i,score_))
        else:
            i += 1
            while score_[i] != 0:
                tag_li.append((key,i,score_))
                i += 1
                if i ==5:
                    break
    return tag_li



  0%|                                                                                        | 0/22710 [00:00<?, ?it/s]

  0%|                                                                              | 32/22710 [00:00<01:11, 317.63it/s]

  0%|▏                                                                             | 58/22710 [00:00<01:16, 296.14it/s]

  0%|▎                                                                             | 89/22710 [00:00<01:15, 298.17it/s]

  1%|▍                                                                            | 121/22710 [00:00<01:14, 302.97it/s]

  1%|▌                                                                            | 151/22710 [00:00<01:14, 301.47it/s]

  1%|▌                                                                            | 181/22710 [00:00<01:16, 295.12it/s]

  1%|▋                                                                            | 211/22710 [00:00<01:16, 295.99it/s]

  1%|▊                        

 17%|████████████▌                                                               | 3756/22710 [00:13<01:04, 293.69it/s]

 17%|████████████▋                                                               | 3788/22710 [00:13<01:02, 300.55it/s]

 17%|████████████▊                                                               | 3819/22710 [00:13<01:03, 299.25it/s]

 17%|████████████▉                                                               | 3849/22710 [00:13<01:04, 293.61it/s]

 17%|████████████▉                                                               | 3880/22710 [00:14<01:03, 296.91it/s]

 17%|█████████████                                                               | 3910/22710 [00:14<01:05, 288.70it/s]

 17%|█████████████▏                                                              | 3941/22710 [00:14<01:03, 294.20it/s]

 17%|█████████████▎                                                              | 3971/22710 [00:14<01:06, 282.85it/s]

 18%|█████████████▍             

 34%|█████████████████████████▌                                                  | 7628/22710 [00:27<00:51, 290.86it/s]

 34%|█████████████████████████▋                                                  | 7659/22710 [00:27<00:50, 295.79it/s]

 34%|█████████████████████████▋                                                  | 7689/22710 [00:27<00:50, 295.58it/s]

 34%|█████████████████████████▊                                                  | 7720/22710 [00:27<00:50, 297.47it/s]

 34%|█████████████████████████▉                                                  | 7750/22710 [00:27<00:50, 296.76it/s]

 34%|██████████████████████████                                                  | 7780/22710 [00:27<00:52, 282.89it/s]

 34%|██████████████████████████▏                                                 | 7809/22710 [00:27<00:53, 277.91it/s]

 35%|██████████████████████████▏                                                 | 7839/22710 [00:27<00:52, 282.06it/s]

 35%|██████████████████████████▎

 51%|██████████████████████████████████████                                     | 11537/22710 [00:40<00:33, 337.89it/s]

 51%|██████████████████████████████████████▏                                    | 11572/22710 [00:40<00:32, 338.80it/s]

 51%|██████████████████████████████████████▎                                    | 11606/22710 [00:40<00:33, 336.42it/s]

 51%|██████████████████████████████████████▍                                    | 11640/22710 [00:40<00:33, 329.08it/s]

 51%|██████████████████████████████████████▌                                    | 11673/22710 [00:41<00:40, 270.66it/s]

 52%|██████████████████████████████████████▋                                    | 11702/22710 [00:41<00:41, 262.25it/s]

 52%|██████████████████████████████████████▊                                    | 11735/22710 [00:41<00:39, 278.28it/s]

 52%|██████████████████████████████████████▊                                    | 11768/22710 [00:41<00:37, 289.21it/s]

 52%|███████████████████████████

 67%|██████████████████████████████████████████████████▎                        | 15219/22710 [00:54<00:26, 285.71it/s]

 67%|██████████████████████████████████████████████████▎                        | 15248/22710 [00:54<00:26, 285.54it/s]

 67%|██████████████████████████████████████████████████▍                        | 15278/22710 [00:54<00:25, 289.21it/s]

 67%|██████████████████████████████████████████████████▌                        | 15308/22710 [00:54<00:26, 281.17it/s]

 68%|██████████████████████████████████████████████████▋                        | 15337/22710 [00:54<00:26, 281.56it/s]

 68%|██████████████████████████████████████████████████▊                        | 15368/22710 [00:54<00:25, 288.19it/s]

 68%|██████████████████████████████████████████████████▊                        | 15400/22710 [00:54<00:24, 294.87it/s]

 68%|██████████████████████████████████████████████████▉                        | 15432/22710 [00:54<00:24, 301.42it/s]

 68%|███████████████████████████

 84%|███████████████████████████████████████████████████████████████▏           | 19145/22710 [01:07<00:11, 297.82it/s]

 84%|███████████████████████████████████████████████████████████████▎           | 19176/22710 [01:07<00:11, 299.06it/s]

 85%|███████████████████████████████████████████████████████████████▍           | 19206/22710 [01:07<00:11, 298.75it/s]

 85%|███████████████████████████████████████████████████████████████▌           | 19237/22710 [01:07<00:11, 300.56it/s]

 85%|███████████████████████████████████████████████████████████████▋           | 19268/22710 [01:07<00:11, 292.49it/s]

 85%|███████████████████████████████████████████████████████████████▋           | 19300/22710 [01:07<00:11, 298.84it/s]

 85%|███████████████████████████████████████████████████████████████▊           | 19332/22710 [01:08<00:11, 304.31it/s]

 85%|███████████████████████████████████████████████████████████████▉           | 19364/22710 [01:08<00:10, 306.47it/s]

 85%|███████████████████████████

,date,id,item,senten,key
0,2020.12.11,황*영,"CKI 가정용 접이식 헤어 드라이어 1200W, CKI-D200, 혼합 색상","[가격, 저렴하다, 간단하다, 손쉽다, 사용, 하다, 아주, 좋다]","[(가성비, 1)]"
1,2020.12.11,황*영,"CKI 가정용 접이식 헤어 드라이어 1200W, CKI-D200, 혼합 색상","[남자, 들, 사용, 하다, 아주, 굿, 이다]",[]
2,2020.12.04,이*정,"CKI 가정용 접이식 헤어 드라이어 1200W, CKI-D200, 혼합 색상","[가격, 대비, 예쁘다, 좋다, !]","[(가성비, 0)]"
3,2020.12.04,황*철,"CKI 가정용 접이식 헤어 드라이어 1200W, CKI-D200, 혼합 색상","[가격, 대비, 만족하다, .]","[(가성비, 0)]"
4,2020.12.01,박*란,"CKI 가정용 접이식 헤어 드라이어 1200W, CKI-D200, 혼합 색상","[저, 는, 아직도, 잘쓰다, 있다]",[]


In [692]:
data_3

,date,id,item,senten,key
0,2020.12.11,황*영,"CKI 가정용 접이식 헤어 드라이어 1200W, CKI-D200, 혼합 색상","[가격, 저렴하다, 간단하다, 손쉽다, 사용, 하다, 아주, 좋다]","[(가성비, 1)]"
1,2020.12.11,황*영,"CKI 가정용 접이식 헤어 드라이어 1200W, CKI-D200, 혼합 색상","[남자, 들, 사용, 하다, 아주, 굿, 이다]",[]
2,2020.12.04,이*정,"CKI 가정용 접이식 헤어 드라이어 1200W, CKI-D200, 혼합 색상","[가격, 대비, 예쁘다, 좋다, !]","[(가성비, 0)]"
3,2020.12.04,황*철,"CKI 가정용 접이식 헤어 드라이어 1200W, CKI-D200, 혼합 색상","[가격, 대비, 만족하다, .]","[(가성비, 0)]"
4,2020.12.01,박*란,"CKI 가정용 접이식 헤어 드라이어 1200W, CKI-D200, 혼합 색상","[저, 는, 아직도, 잘쓰다, 있다]",[]
...,...,...,...,...,...
22705,2019.01.10,김*지,"비달사순 파워 블랙앤로즈 드라이어 VSD5129K 2000W, 로즈골드","[완전, 좋다]","[(냉풍 , 1)]"
22706,2019.01.10,김*지,"비달사순 파워 블랙앤로즈 드라이어 VSD5129K 2000W, 로즈골드","[맘, 에, 들다]",[]
22707,2019.01.03,김*환,"비달사순 파워 블랙앤로즈 드라이어 VSD5129K 2000W, 로즈골드","[바람, 조절, 버튼, 이, 똑, 딱이다, 잘못, 건들, 이면, 바뀌다, 불편하다,...","[(바람 세기, 1), (바람 세기, 2), (바람 세기, 3), (풍량 조절 , ..."
22708,2018.12.31,권*영,"비달사순 파워 블랙앤로즈 드라이어 VSD5129K 2000W, 로즈골드","[빠르다, 배송, 에, 대다, 만족하다, .]",[]


## funda, inter, key 에서 불필요 한거 지우기 

In [594]:
for key,value in a_dic.items():
    print()
    
    print(value["funda"])
    print(value["noun_inter"])
    print(value["key"])
    
    print()


[['바람', '파워'], ['바람', '세기'], ['드라이기', '파워'], ['바람', '강도'], ['건조', '속도'], ['바람', '강하다'], ['바람', '세다'], ['바람', '쎄다'], ['바람', '약하다'], ['건조', '빠르다'], ['건조', '느리다'], ['건조', '빨르다']]
[['세기'], ['느낌'], ['정도'], ['바람']]
[['바람']]


[['풍량', '조절'], ['바람', '세기', '조절'], ['풍속', '조절'], ['강도', '조절'], ['바람강도', '조절'], ['바람', '강도', '조절']]
[['세기'], ['것'], ['강도'], ['강약'], ['바람'], ['조절'], ['단계']]
[['바람']]


[['바람', '회전'], ['회전', '바람'], ['토네이도']]
[['토네이도']]
[['바람']]


[['냉풍'], ['찬바람'], ['차가운', '바람'], ['냉', '풍'], ['차갑다', '바람'], ['찬', '바람'], ['바람', '차갑다']]
[['냉'], ['완전'], ['바람'], ['냉풍']]
[['온도']]


[['따뜻한', '바람'], ['부드러운', '바람'], ['온풍'], ['온', '풍'], ['따뜻하다', '바람'], ['부드럽다', '바람']]
[['손상'], ['온도'], ['별로'], ['바람']]
[['온도']]


[['열풍'], ['뜨거운', '바람'], ['뜨겁다', '바람']]
[['바람']]
[['온도']]


[['온도', '3단', '조절'], ['온도', '2단계'], ['온도', '3단계'], ['온도', '조절', '단계'], ['온도', '조절'], ['온도', '3', '단'], ['온도', '2', '단'], ['온도', '단계']]
[['온도'], ['조절']]
[['온도']]


[['온도조절', '슬라이드'], ['푸쉬락', '버튼'], ['슬라이드식'], ['슬라이드', '식']]
[['스위치'], [

In [638]:
for key,value in a_dic.items():
    print()
    print(value["key"])
    print(value["key2"])
    


[['바람']]
[['바람']]

[['바람']]
[['바람']]

[['바람']]
[['바람']]

[['온도']]
[['온도']]

[['온도']]
[['온도']]

[['온도']]
[['온도']]

[['온도']]
[['온도']]

[['조작', '감']]
[['조작'], ['감']]

[['조작', '감']]
[['조작'], ['감']]

[['조작', '감']]
[['조작'], ['감']]

[['조작', '감']]
[['조작'], ['감']]

[['디지털', '센서']]
[['디지털'], ['센서']]

[['디지털', '센서']]
[['디지털'], ['센서']]

[['디지털', '센서']]
[['디지털'], ['센서']]

[['발생', '성분']]
[['발생'], ['성분']]

[['발생', '성분']]
[['발생'], ['성분']]

[['발생', '성분']]
[['발생'], ['성분']]

[['발생', '성분']]
[['발생'], ['성분']]

[['안전', '기능']]
[['안전'], ['기능']]

[['안전', '기능']]
[['안전'], ['기능']]

[['안전', '기능']]
[['안전'], ['기능']]

[['안전', '기능']]
[['안전'], ['기능']]

[['안전', '기능']]
[['안전'], ['기능']]

[['위생']]
[['위생']]

[['위생']]
[['위생']]

[['스타', '일링', '부속품']]
[['스타'], ['일링'], ['부속품']]

[['스타', '일링', '부속품']]
[['스타'], ['일링'], ['부속품']]

[['스타', '일링', '부속품']]
[['스타'], ['일링'], ['부속품']]

[['스타', '일링', '부속품']]
[['스타'], ['일링'], ['부속품']]

[['스타', '일링', '부속품']]
[['스타'], ['일링'], ['부속품']]

[['손잡이', '형태']]
[['손잡이'], ['형태']]

[['손잡이', '형태']]
[['손잡이

In [595]:
removal2 = [['기능'],['감'], ['/'], ['및']]

In [604]:
tester_ = a_dic["프리미엄 "]["noun_inter"]
tester_

[['돈']]

In [597]:
for re in removal:
    try:
        tester_.remove(re)
    except:
        pass
tester_

[['돈']]

In [ ]:
dic_1 = a_dic
def evalue()
    for key,value in dic_1.items():
        for sen in sens:
            sco_1 = sum(list(map(lambda x:match_f(x,sen)[0] ,value["funda"])))
            sco_2 = sum(list(map(lambda x:match_f(x,sen)[0] ,value["noun_inter"])))
            sco_3 = sum(list(map(lambda x:match_f(x,sen)[0] ,value["key"])))

            sco_4 = sum(list(map(lambda x:match_f(x,sen)[0] ,value["pos"])))
            sco_5 = sum(list(map(lambda x:match_f(x,sen)[0] ,value["neg"])))

            score_ = [sco_1,sco_2,sco_3,sco_4,sco_5]
            total_sum = sum(score_)
            if sco_1 != 0:
                print(key)
                print(sen)
                print(score_)
                #print(key,total_sum,sen)
                print(list(map(lambda x:match_f(x,sen)[1] ,value["funda"])))
            elif sco_2 != 0:
                print(key)
                print(sen)
                print(score_)
                #print(key,total_sum,sen)
                print(list(map(lambda x:match_f(x,sen)[1] ,value["noun_inter"])))

# Data

In [17]:
blog_li = data_list(r"C:\Users\Samsung\jupyter\Project_LG_textmining\data\hair\blog")
coo_li = data_list(r"C:\Users\Samsung\jupyter\Project_LG_textmining\data\hair\shopping\coupang")
auc_li = data_list(r"C:\Users\Samsung\jupyter\Project_LG_textmining\data\hair\shopping\auction")

In [20]:
data = load_all(coo_li,[0,10])
data_2 = all_pro(data)

 32%|███████████████████████▊                                                  | 3270/10142 [00:00<00:00, 32478.11it/s]

sentence_split


  0%|▏                                                                             | 30/10142 [00:00<00:33, 298.01it/s]

pos_tagging


100%|███████████████████████████████████████████████████████████████████████████| 10142/10142 [00:33<00:00, 302.07it/s]


In [21]:
data_2.head()

,date,id,item,score,contents,liked,parse_date,year,month,day,sentences,sen_tag,sen_word
0,2020.12.11,황*영,"CKI 가정용 접이식 헤어 드라이어 1200W, CKI-D200, 혼합 색상",5,가격저렴하고 간단하고 손쉽게 사용할수있어 아주 좋습니다 남자들 사용하기 아주 굿입니다,0,2020-12-11,2020,2020-12,2020-12-11,"[가격저렴하고 간단하고 손쉽게 사용할수있어 아주 좋습니다, 남자들 사용하기 아주 굿...","[[(가격, Noun), (저렴하다, Adjective), (간단하다, Adject...","[[가격, 저렴하다, 간단하다, 손쉽다, 사용, 하다, 아주, 좋다], [남자, 들..."
1,2020.12.04,이*정,"CKI 가정용 접이식 헤어 드라이어 1200W, CKI-D200, 혼합 색상",5,가격대비 예쁘고 좋아요!,0,2020-12-04,2020,2020-12,2020-12-04,[가격대비 예쁘고 좋아요!],"[[(가격, Noun), (대비, Noun), (예쁘다, Adjective), (좋...","[[가격, 대비, 예쁘다, 좋다, !]]"
2,2020.12.04,황*철,"CKI 가정용 접이식 헤어 드라이어 1200W, CKI-D200, 혼합 색상",4,가격대비 만족합니다.,0,2020-12-04,2020,2020-12,2020-12-04,[가격대비 만족합니다.],"[[(가격, Noun), (대비, Noun), (만족하다, Adjective), (...","[[가격, 대비, 만족하다, .]]"
3,2020.12.01,박*란,"CKI 가정용 접이식 헤어 드라이어 1200W, CKI-D200, 혼합 색상",5,저는 아직도 잘쓰고 있어요 산지 좀 되어ㅛ는데도 좋네요\n가볍고,0,2020-12-01,2020,2020-12,2020-12-01,"[저는 아직도 잘쓰고 있어요, 산지 좀 되어ㅛ는데도 좋네요, 가볍고]","[[(저, Noun), (는, Josa), (아직도, Adverb), (잘쓰다, V...","[[저, 는, 아직도, 잘쓰다, 있다], [산지, 좀, 되어다, ㅛ, 는데도, 좋다..."
4,2020.11.30,유*환,"CKI 가정용 접이식 헤어 드라이어 1200W, CKI-D200, 혼합 색상",5,약간 탄내가 조금 나요,0,2020-11-30,2020,2020-11,2020-11-30,[약간 탄내가 조금 나요],"[[(약간, Noun), (타다, Verb), (조금, Noun), (나요, Eom...","[[약간, 타다, 조금, 나요]]"


In [666]:
date_ = []
id_ = []
product_ = []
sen_ = []
for i in range(len(data_2)):
    for sen in data_2.loc[i,"sen_word"]:
        date_.append(data_2.loc[i,"date"])
        id_.append(data_2.loc[i,"id"])
        product_.append(data_2.loc[i,"item"])
        sen_.append(sen)
data_3 = pd.DataFrame([date_,id_,product_,sen_]).T.rename(columns={0:"date",1:"id",2:"item",3:"senten"})
data_3.head()

,date,id,item,senten
0,2020.12.11,황*영,"CKI 가정용 접이식 헤어 드라이어 1200W, CKI-D200, 혼합 색상","[가격, 저렴하다, 간단하다, 손쉽다, 사용, 하다, 아주, 좋다]"
1,2020.12.11,황*영,"CKI 가정용 접이식 헤어 드라이어 1200W, CKI-D200, 혼합 색상","[남자, 들, 사용, 하다, 아주, 굿, 이다]"
2,2020.12.04,이*정,"CKI 가정용 접이식 헤어 드라이어 1200W, CKI-D200, 혼합 색상","[가격, 대비, 예쁘다, 좋다, !]"
3,2020.12.04,황*철,"CKI 가정용 접이식 헤어 드라이어 1200W, CKI-D200, 혼합 색상","[가격, 대비, 만족하다, .]"
4,2020.12.01,박*란,"CKI 가정용 접이식 헤어 드라이어 1200W, CKI-D200, 혼합 색상","[저, 는, 아직도, 잘쓰다, 있다]"


# evalue_model

In [22]:
a_dic

{'바람 세기': {'pos': [['바람', '쎄다'],
   ['드라이기', '바람', '세기', '만족하다'],
   ['바람', '세다', '좋다'],
   ['바람', '세기', '좋다'],
   ['바람', '세기', '맘에듭니'],
   ['바람', '세기', '강하다'],
   ['강하다', '바람', '젖다', '머리', '금방', '말', '좋다'],
   ['바람', '세다', '빨리', '말르다'],
   ['사용', '하다', '적당하다', '바라다', '이다'],
   ['바람', '세기', '충분하다'],
   ['바람', '세기', '단계', '별로', '있다'],
   ['바람', '쎄다', '좋다'],
   ['바람', '세기', '최고', '이다'],
   ['바람', '세다', '빨리', '건조', '되다', '만족하다'],
   ['바람', '쎄다', '물기', '금방', '마르다', '느낌'],
   ['바람', '세기', '흡족하다'],
   ['바람', '쎄다', '따라오다', '힘들껍니'],
   ['바람', '최상급'],
   ['강력하다', '바라다', '이다'],
   ['빠르다', '확실하다', '건조', '가능하다'],
   ['바람', '세기', '엄청나다'],
   ['빠르다', '건조', '만족하다'],
   ['출력', '그렇다', '바람', '완전', '쎄다'],
   ['파워', '풀', '바람', '좋다'],
   ['바람', '세기', '최고', '진짜', '속', '시원하다', '나오다'],
   ['바람', '세기', '장난', '아니다', '쩌렁'],
   ['바람', '세기', '더하다', '나위', '없다'],
   ['바람', '세기', '작살', '여'],
   ['역시', '가용', '바람', '세기', '다르다'],
   ['가정', '용', '확실하다', '쎄다', '미용실', '정도', '바람', '나오다'],
   ['바람', '세기', '남다름']],
  'neg': [